# Implementing GPT-2 architecture

In [ ]:
# gpt 2 config
GPT_2_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "embedding_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [ ]:
# import pytorch
import torch
import torch.nn as nn
import tiktoken

In [ ]:
# GPT-2 placeholder
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_emb = nn.Embedding(cfg["vocab_size"], cfg["embedding_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["embedding_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.tsf_blocks = nn.Sequential(*[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["embedding_dim"])
        self.out_heads = nn.Linear(cfg["embedding_dim"], cfg["vocab_size"], bias = False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        token_embeds = self.token_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
        x = token_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.tsf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_heads(x)
        return logits

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x

class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps = 1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [ ]:
# sample input
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim = 0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
# instance of GPT class
model = DummyGPTModel(GPT_2_CONFIG)
logits = model(batch)
print("Logits shape:", logits.shape)
print(logits)

Logits shape: torch.Size([2, 4, 50257])
tensor([[[ 0.0826, -0.2156,  1.6275,  ..., -1.2274, -0.6845, -0.8324],
         [ 0.1974,  0.6233, -0.1014,  ...,  0.1794,  1.2948,  2.0486],
         [ 0.2494,  1.9467,  1.1572,  ..., -0.0411,  2.2900,  0.6426],
         [-0.3342,  0.5649,  0.5333,  ...,  0.2540,  0.3622, -0.3424]],

        [[ 0.0602, -0.2113,  1.6655,  ..., -0.9637, -0.6737, -0.9095],
         [ 1.3411,  0.9896, -0.0440,  ...,  0.0917,  1.0057, -0.2773],
         [-0.4158,  1.5241,  0.5208,  ...,  0.3793,  1.0881, -0.5831],
         [ 0.9586, -1.3280,  0.5330,  ...,  0.2154, -0.1688, -0.1250]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
# layer normalization class
class LayerNorm(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(embedding_dim))
        self.shift = nn.Parameter(torch.ones(embedding_dim))

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        variance = x.var(dim = -1, keepdim = True, unbiased = True)
        normalized_x = (x - mean) / torch.sqrt(variance + self.eps)
        return self.scale * normalized_x + self.shift

In [ ]:
batch_example = torch.randn(2, 5)
print(batch_example)

tensor([[-0.6206,  0.7020, -2.6237, -0.6853,  0.3450],
        [ 0.5030, -0.7696,  0.8332,  2.0799,  1.1490]])


In [ ]:
ln = LayerNorm(embedding_dim = 5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim = -1, keepdim = True)
var = out_ln.var(dim = -1, keepdim = True, unbiased = True)
print(mean)
print(var)

tensor([[1.],
        [1.]], grad_fn=<MeanBackward1>)
tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


## Implementing Feed Forward NN with GELU activation function

In [ ]:
# GELU activation function
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

In [ ]:
# feed forward layer
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
        nn.Linear(cfg["embedding_dim"], 4 * cfg["embedding_dim"]),
        GELU(),
        nn.Linear(4 * cfg["embedding_dim"], cfg["embedding_dim"]),
    )
    def forward(self, x):
        return self.layers(x)

In [ ]:
ffn = FeedForward(GPT_2_CONFIG)
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


## Shortcut connections

In [ ]:
# class
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcuts):
        super().__init__()
        self.use_shortcut = use_shortcuts
        self.layers = nn.ModuleList([
            # Implement 5 layers
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

    def forward(self, x):
        for layer in self.layers:
            layer_output = layer(x)
            if self.use_shortcut and x.shape == layer.output.shape:
                x += layer_output
            else:
                x = layer_output
        return x

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = torch.nn.Linear(d_in, d_out, qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)
        self.dropout = torch.nn.Dropout(p = 0.5)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal = 1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [ ]:
# transformer block of GPT-2
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = cfg["embedding_dim"],
            d_out =  cfg["embedding_dim"],
            context_length = cfg["context_length"],
            dropout = cfg["drop_rate"],
            num_heads = cfg["n_heads"],
            qkv_bias = cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["embedding_dim"])
        self.norm2 = LayerNorm(cfg["embedding_dim"])
        self.drop = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop(x)
        x += shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop(x)
        x += shortcut
        return x

In [ ]:
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_2_CONFIG)
output = block(x)
print(output)

tensor([[[ 0.1890,  1.6600,  0.2652,  ..., -0.8592, -0.4959,  1.1356],
         [-0.2148,  1.8773,  2.0456,  ...,  1.5177,  0.1359,  1.9233],
         [ 1.4081,  0.1454,  0.3777,  ...,  1.1837,  0.0310,  1.7865],
         [-0.1163,  1.0859,  0.4263,  ...,  0.7645,  0.7893,  1.1430]],

        [[ 0.0382, -1.0362, -0.0363,  ...,  0.6393, -0.1076,  0.1608],
         [ 0.6126,  0.0068,  0.7614,  ...,  1.3073,  0.3980,  1.3227],
         [ 0.2942,  0.2697,  1.2901,  ...,  0.9598,  0.6771,  0.4501],
         [ 0.1525, -0.5742,  0.2605,  ...,  0.8544,  0.2254,  1.6791]]],
       grad_fn=<AddBackward0>)


In [ ]:
# final GPT model
class GPTmodel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_emb = nn.Embedding(cfg["vocab_size"], cfg["embedding_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["embedding_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_block = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["embedding_dim"])
        self.out_head = nn.Linear(cfg["embedding_dim"], cfg["vocab_size"], bias = False)

    def forward(self, i_idx):
        batch_size, seq_len = i_idx.shape
        token_embeds = self.token_emb(i_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = i_idx.device))
        x = token_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_block(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
# instance
torch.manual_seed(123)
model = GPTmodel(GPT_2_CONFIG)
output = model(batch)
print(output)
print("Input batch:", batch)
print("Output shape:", output.shape)

tensor([[[ 0.5514, -0.9670,  0.7579,  ..., -0.6175, -1.3106, -0.0158],
         [ 0.7144, -1.6823, -0.0286,  ..., -0.1798, -0.9726,  0.0926],
         [ 0.5534, -1.5468, -0.1502,  ..., -0.1483, -1.8168, -0.6314],
         [-0.8051, -1.6082,  0.0258,  ...,  0.2877, -1.0677,  0.3591]],

        [[-0.9335, -1.4204,  0.3273,  ..., -0.4144, -1.3657, -0.0283],
         [-0.2963, -1.4595, -0.3406,  ..., -0.1631, -1.3662,  0.4405],
         [ 0.8794, -0.8463,  0.0735,  ...,  0.3200, -1.7805,  0.0260],
         [-0.1378, -1.1232,  0.5363,  ...,  1.0011, -1.5238, -0.1419]]],
       grad_fn=<UnsafeViewBackward0>)
Input batch: tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
Output shape: torch.Size([2, 4, 50257])


In [ ]:
# parameter size
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters:", total_params)

Total Parameters: 163009536


In [ ]:
# storage size
total_size_bytes = total_params * 4
total_size = total_size_bytes / (1024 * 1024)
print(f"Total size: {total_size:.2f} MB")

Total size: 621.83 MB


## Generating text

In [ ]:
# function
def generate_text(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]
        probs = torch.softmax(logits, dim = -1)
        idx_next = torch.argmax(probs, dim = -1, keepdim = True)
        idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print(encoded)
print(encoded_tensor.shape)

[15496, 11, 314, 716]
torch.Size([1, 4])


In [ ]:
model.eval()
out = generate_text(model, encoded_tensor, max_new_tokens = 6, context_size = GPT_2_CONFIG["context_length"])
print(out)

tensor([[15496,    11,   314,   716, 35482]])


In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am standout
